# *An Example With ScaPy*

ScaPy is a library that can be used to create and manipulate network packets, but it is also capable of doing minor attacks as well. There are going to be a few examples of what ScaPy can do.

- Creating basic packets
- tracerouting 
- attacks   



In [ ]:
from scapy.all import *

# define the target ip and port
target_ip = "127.0.0.1"
target_port = 80

# create the packet
packet = IP(dst=target_ip)/TCP(dport=target_port, flags="S")

send(packet)

print(f"Sent SYN packet to {target_ip} on port {target_port}")

OSError: [Errno 100] Network is down